In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy.stats import norm
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
%matplotlib inline
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

import os
print(os.listdir("../House-prices-advanced-regression-techniques/all"))

# Any results you write to the current directory are saved as output.

['test.csv.gz', 'sample_submission.csv.gz', 'test.csv', 'data_description.txt', 'train.csv', 'train.csv.gz', 'sample_submission.csv']


In [3]:
df_train = pd.read_csv('../House-prices-advanced-regression-techniques/all/train.csv')
df_train.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [4]:
#descriptive statistics summary
df_train['SalePrice'].describe()

count      1460.000000
mean     180921.195890
std       79442.502883
min       34900.000000
25%      129975.000000
50%      163000.000000
75%      214000.000000
max      755000.000000
Name: SalePrice, dtype: float64

In [5]:
# Remove Missing Values

# missing data  Percent = Number of total missing values for each column / Number of total values

total = df_train.isnull().sum().sort_values(ascending=False)
percent = (df_train.isnull().sum()/df_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

In [6]:
df_train = df_train.drop((missing_data[missing_data['Total'] > 1]).index,1)
df_train = df_train.drop(df_train.loc[df_train['Electrical'].isnull()].index)
df_train.isnull().sum().max() #just checking that there's no missing data missing...

0

In [7]:
# Log Transform the values to have linear relationship

numeric_feats = df_train.dtypes[df_train.dtypes != "object"].index
skewed_feats = df_train[numeric_feats].apply(lambda x: (x.dropna().skew())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index

df_train[skewed_feats] = np.log1p(df_train[skewed_feats])


In [8]:
df_train = pd.get_dummies(df_train)
 
#filling NA's with the mean of the column:
df_train = df_train.fillna(df_train.mean())

In [9]:
# create X Train and Y Train and test set from training data
# col 34 contains y

X_train = pd.concat([df_train.iloc[:,0:34].copy(),df_train.iloc[:,35:].copy()],axis=1)[0:1167]
y_train = df_train.iloc[:,34].copy()[0:1167] 

X_test = pd.concat([df_train.iloc[:,0:34].copy(),df_train.iloc[:,35:].copy()],axis=1)[1167:]
y_test = df_train.iloc[:,34].copy()[1167:] 
# df_train.columns.get_loc("SalePrice")

In [10]:
from sklearn import  linear_model

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
Y_pred = regr.predict(X_test)

In [11]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_true=y_test, y_pred=Y_pred) 

0.02504569024862892

In [12]:
# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, Y_pred))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, Y_pred))

Coefficients: 
 [-2.93956696e-06 -1.34211958e-02  9.89738111e-02  4.21028727e-02
  3.93987977e-02  2.00039745e-03  1.09407859e-03  7.82249441e-03
 -1.67645404e-03 -5.39206774e-03  1.70194277e-02  5.97640524e-02
 -4.72491589e-03 -7.22633529e-03  4.26779590e-01  3.78106113e-02
  2.26797309e-02  3.24790326e-02  2.12675786e-02 -1.26192327e-02
 -1.49719020e-01  6.84873411e-03  2.22657003e-02  1.06859457e-02
  1.19355328e-04  3.09219610e-03  6.51881362e-04  3.34317248e-03
 -1.80215027e-03  9.76878906e-03  3.47984401e-03 -2.38860460e-03
 -2.62788926e-03 -4.68844838e-03 -3.46810670e-01  1.02144603e-01
  8.94087164e-02  9.32642904e-02  6.19930600e-02 -4.33180076e-02
  4.33180076e-02 -2.38477415e-02 -9.11436587e-03  4.44107076e-02
 -1.14486001e-02 -2.55153937e-02  1.36948745e-02 -3.11778986e-03
  1.49383090e-02  9.27270969e-02 -9.27270969e-02  2.52548055e-02
  3.92813868e-02 -7.94531282e-03 -7.46632189e-02  1.80723395e-02
  1.07497255e-02  6.43882079e-02 -7.51379334e-02  2.88583657e-02
  4.45133

In [13]:
np.sqrt(mean_squared_error(y_test, Y_pred))
 

0.15825830230553126

In [151]:
df_test = pd.read_csv('../House-prices-advanced-regression-techniques/all/test.csv')

In [152]:
df_test.shape

(1459, 80)

In [14]:
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [43]:
# set up random forest regressor to find the feature importances
forestreg = RandomForestRegressor(max_depth=99, n_estimators=2000, random_state=0).fit(X_train, y_train)
Y_pred_forestreg = forestreg.predict(X_test)


In [44]:
mean_squared_error(y_true=y_test, y_pred=Y_pred_forestreg) 

0.021846840674685914

In [45]:
lassoreg = Lasso(alpha=0.001).fit(X_train, y_train)
Y_pred_lassoreg = lassoreg.predict(X_test)
mean_squared_error(y_true=y_test, y_pred=Y_pred_lassoreg)

0.022284733905649667

In [47]:
ridgereg = Ridge(alpha=10).fit(X_train, y_train)
Y_pred_ridgereg = ridgereg.predict(X_test)
mean_squared_error(y_true=y_test, y_pred=Y_pred_ridgereg)

0.022969219506477546